In [9]:
#downloading and unzipping the IMDB reviews data

!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

In [12]:
# There’s also a train/unsup subdirectory in there, which we don’t need. Let’sdelete it:
!rm -r aclImdb/train/unsup

Next, let’s prepare a validation set by setting apart 20% of the training text files in a
 new directory, aclImdb/val:

In [17]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir/'val'
train_dir = base_dir/'train'

for category in ("neg", "pos"):
    os.makedirs(val_dir/category)
    files = os.listdir(train_dir/category)
    random.Random(1327).shuffle(files)
    num_val_samples = int(0.2*len(files))
    val_files = files[-num_val_samples: ]
    for fname in val_files:
        shutil.move(train_dir/category/fname, 
                    val_dir/category/fname)

In [1]:
from tensorflow import keras
batch_size = 32

train_dataset = keras.utils.text_dataset_from_directory(
    "aclImdb/train", 
    batch_size=batch_size)

validation_dataset = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size)

test_dataset = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size)


Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [2]:
for inputs, targets in train_dataset:
    print('shape', inputs.shape)
    print('datatype', inputs.dtype)
    print('targets shape', targets.shape)
    print('datatype', targets.shape)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets)
    break

shape (32,)
datatype <dtype: 'string'>
targets shape (32,)
datatype (32,)
inputs[0]: tf.Tensor(b'Interesting story about a soldier in a war who misses out on saving the life of a young girl from the enemy and is haunted by this event, even though he did save many other captive children. The film flashes a head and this soldier is now a teacher in a high school that is managed mostly by policemen patrolling the hallways, bathrooms and even class rooms. In other words, the High School is a prison and most of the kids pay very little attention to their teachers or principal. Dolph Lundgren,(Sam Decker) plays the soldier/school teacher and decides he is going to quit teaching and go into another field. However, the principal asks him to have a Detention Class as his last duty as a teacher. It is at this point in the film when all Hell breaks loose and the story becomes a complete BOMB. Try to enjoy it, if you decided to View IT !', shape=(), dtype=string)
targets[0]: tf.Tensor([0 1 0 1 0 0

**Preprocessing our datasets with a TextVectorization layer**

In [7]:
from keras.layers import TextVectorization

text_vectorization = TextVectorization(
    max_tokens=20000, 
    ngrams=2,
    output_mode='multi_hot')

In [8]:
text_only_train_ds = train_dataset.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

binary_2gram_train_ds = train_dataset.map(
        lambda x, y: (text_vectorization(x), y),
        num_parallel_calls=4
)

binary_2gram_validation_dataset = validation_dataset.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4
)

binary_2gram_test_dataset = test_dataset.map(
    lambda x, y: (text_vectorization(x), y)
)

In [13]:
for a, b in binary_2gram_test_dataset:
    print(a[1])
    print(b[0])
    break

tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
tf.Tensor(0, shape=(), dtype=int32)


In [14]:
#model building and compilation
#Training and testing the binary unigram model
from tensorflow import keras
from tensorflow.keras import layers


def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens, ))
    x = layers.Dense(hidden_dim, activation='relu')(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = keras.Model(inputs, outputs)
    model.compile(optimizer='rmsprop',
                 loss='binary_crossentropy',
                 metrics=["accuracy"])
    return model

In [15]:
#instantiating the model from get_model 

model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
#callbacks = ModelCheckpoint("binary_1gram.keras", 
#                             save_best_only=True)


In [17]:
# from tensorflow.keras.callbacks import ModelCheckpoint


model.fit(binary_2gram_train_ds.cache(),
         validation_data=binary_2gram_validation_dataset.cache(),
         epochs=10)

Epoch 1/10
625/625 [==============================] - 25s 39ms/step - loss: 0.3747 - accuracy: 0.8423 - val_loss: 0.2546 - val_accuracy: 0.8988
Epoch 2/10
625/625 [==============================] - 8s 12ms/step - loss: 0.2374 - accuracy: 0.9157 - val_loss: 0.2605 - val_accuracy: 0.9032
Epoch 3/10
625/625 [==============================] - 6s 10ms/step - loss: 0.2025 - accuracy: 0.9330 - val_loss: 0.2766 - val_accuracy: 0.9044
Epoch 4/10
625/625 [==============================] - 6s 10ms/step - loss: 0.1863 - accuracy: 0.9446 - val_loss: 0.2950 - val_accuracy: 0.9062
Epoch 5/10
625/625 [==============================] - 6s 10ms/step - loss: 0.1802 - accuracy: 0.9484 - val_loss: 0.3139 - val_accuracy: 0.9012
Epoch 6/10
625/625 [==============================] - 6s 10ms/step - loss: 0.1730 - accuracy: 0.9527 - val_loss: 0.3200 - val_accuracy: 0.8990
Epoch 7/10
625/625 [==============================] - 6s 10ms/step - loss: 0.1630 - accuracy: 0.9550 - val_loss: 0.3409 - val_accuracy: 0.897

In [19]:
print(f"Test acc: {model.evaluate(binary_2gram_test_dataset)[1]:.3f}")

782/782 [==============================] - 78s 99ms/step - loss: 0.3776 - accuracy: 0.8888
Test acc: 0.889


**So, our model achieved the accuracy of 90%** 
We can still imporove this applying some other techniques.